In [4]:
import pandas as pd
import numpy as np
import os
import sqlite3
import shutil
import datetime
import matplotlib.pyplot as plt
import tabulate
from IPython.display import HTML, display, Markdown
import csv
import unicodedata
import getopt
from ipywidgets import HBox, VBox, Layout

import ipywidgets as widgets
from ipywidgets import HBox, VBox, Layout
import graphviz
from GraphVizUtil import *
from GraphVizFormats import *
import warnings
warnings.filterwarnings('ignore')
import seaborn as sb
sb.set(style='darkgrid', font_scale=1.2)

def filter_descriptions(tech_comm_desc):
    try:
        tech_comm_desc = tech_comm_desc.values[0][0].replace('#', '').replace('"','').replace("\n",'').strip()
    except:
        tech_comm_desc = 'No description provided'
    return tech_comm_desc

def create_args_flowd(df_graph):
    nodes, tech, ltech, to_tech, from_tech = set(), set(), set(), set(), set()
    for ind,row in df_graph.iterrows():
        #descriptions:
        input_comm_des = filter_descriptions(pd.read_sql("SELECT comm_desc FROM commodities WHERE comm_name='" + row['input_comm'] + "'", con))
        output_comm_des = filter_descriptions(pd.read_sql("SELECT comm_desc FROM commodities WHERE comm_name='" + row['output_comm'] + "'", con))
        tech_des = filter_descriptions(pd.read_sql("SELECT tech_desc FROM technologies WHERE tech='" + row['tech'] + "'", con))

        if 'ethos' in row['input_comm']:
            ltech.add('"' + row['tech'] + '"' +  ' [tooltip = "' + tech_des + '"]')
        else :
            nodes.add('"' + row['input_comm'] + '"' +  ' [tooltip = "' + input_comm_des + '"]')
        nodes.add('"' + row['output_comm'] + '"' +  ' [tooltip = "' + output_comm_des + '"]')
        tech.add('"' + row['tech'] + '"' +  ' [tooltip = "' + tech_des + '"]')

        if row['input_comm'] != 'ethos':
            to_tech.add('"%s"' % row['input_comm'] + '\t->\t"%s"' % row['tech']) 
        from_tech.add('"%s"' % row['tech'] + '\t->\t"%s"' % row['output_comm'])
    args = dict(
    enodes = "".join('%s;\n\t\t' % x for x in nodes),
    tnodes = "".join('%s;\n\t\t' % x for x in tech),
    iedges = "".join('%s;\n\t\t' % x for x in to_tech),
    oedges = "".join('%s;\n\t\t' % x for x in from_tech),
    snodes = ";".join('%s' %x for x in ltech),
    )
    return args


def return_format_colors():
    colors = {}
    colors.update(getColorConfig(False))
    return colors, quick_run_dot_fmt

def return_flowd_table(final_dem, level=1):
    df = pd.read_sql("SELECT * FROM Efficiency", con)
    df_sel = df[df['output_comm']==final_dem]
    if len(df_sel)==0:
        df_sel = df[df['tech']==final_dem]
    inputs = df_sel['input_comm'].unique()
    iterval=0
    if level!=0:
        while len(inputs)>0:
            df_append = df[df['output_comm'].isin(inputs)]
            df_sel = pd.concat([df_sel, df_append])
            inputs = df_append['input_comm'].unique()
            iterval+=1
            if iterval>level-1:
                break
    df_graph = df_sel[['input_comm', 'tech', 'output_comm']].drop_duplicates()
    return df_graph

def return_flowd_table_fwds(final_dem):
    df = pd.read_sql("SELECT * FROM Efficiency", con)
    df_sel = df[df['output_comm']==final_dem]
    if len(df_sel)==0:
        df_sel = df[df['tech']==final_dem]
    inputs = df_sel['input_comm'].unique()
    outputs = df_sel['output_comm'].unique()

    iterval=0
    while len(inputs)>0:
        df_append = df[df['output_comm'].isin(inputs)]
        df_sel = pd.concat([df_sel, df_append])
        inputs = df_append['input_comm'].unique()
        iterval+=1
        if iterval>2:
            break
    iterval=0
    while len(outputs)>0:
        df_append = df[df['input_comm'].isin(outputs)]
        df_sel = pd.concat([df_sel, df_append])
        outputs = df_append['output_comm'].unique()
        iterval+=1
        if iterval>=0:
            break

    df_graph = df_sel[['input_comm', 'tech', 'output_comm']].drop_duplicates()
    return df_graph

con = sqlite3.connect(r'../US_9R_8D.sqlite') #change path to database
cur = con.cursor()   
con.text_factory = str 

def controls_rows(w):
    controls = HBox(w.children[:-1], layout = Layout(flex_flow='row wrap', width='max-content'))
    output = w.children[-1]
    display(VBox([controls, output],  layout = Layout(flex_flow='columns wrap', width='max-content', size=10)))

### Carbon Dioxide Removal (CDR)  Overview <a class="anchor" id="CDR"></a>
Technologies broadly include direct air capture and geologic sequestration and storage.



#### Direct air capture <a class="anchor" id="DAC"></a>
We model two direct air capture technologies based on <a href="https://www.cell.com/joule/fulltext/S2542-4351(18)30225-3#tbl2" rel="nofollow">Keith et al. (2018)</a>. One technology converts captured CO<sub>2</sub> into synthetic fuels, while the other sequesters it. Table 2 in Keith et al. outlines key technoeconomic parameters for 5 direct air capture systems, plants A, B, C, D, and D'. We model plants A, B, and D. Capital costs, fixed O&M costs, and efficiencies for the DAC to synthetic fuel pathway are based on plant D. Technoeconomic parameters for the DAC to sequestration pathway are based on plant A in 2030, with costs decreasing linearly to plant B's costs by 2050. Plant A and plant B are identical, plant B merely represents a more mature, and therefore less expensive, technology. 

The DAC to sequestration technology has only natural gas as an energy input, while the synthetic fuels pathway requires both natural gas and electricity. 

**Table 1.** Key DAC parameters

|Technology|Parameter | Value | Units | Source |
| :----- | :----- | :-- |:-- |:--------- |
|Synthetic fuels|Capital Cost| 609  | \$/t-CO<sub>2</sub>/year| <a href="https://www.cell.com/joule/fulltext/S2542-4351(18)30225-3#tbl2" rel="nofollow">Keith et al. (2018)</a>|
||Variable O&M Cost| 23 | \$/t-CO<sub>2</sub>| <a href="https://www.cell.com/joule/fulltext/S2542-4351(18)30225-3#tbl2" rel="nofollow">Keith et al. (2018)</a>|
||Energy Consumption| 5.25 / 77  |PJ NG / kWh electricity| <a href="https://www.cell.com/joule/fulltext/S2542-4351(18)30225-3#tbl2" rel="nofollow">Keith et al. (2018)</a>,  <a href="https://pubs.rsc.org/en/content/articlelanding/2020/SE/c9se00479c#!divAbstractlife" rel="nofollow">Liu et al.</a>|
|Sequestration|Capital Cost| 1146 (2030) - 793 (2050)  | \$/t-CO<sub>2</sub>/year| <a href="https://www.cell.com/joule/fulltext/S2542-4351(18)30225-3#tbl2" rel="nofollow">Keith et al. (2018)</a>|
||Variable O&M Cost| 42 (2030) - 30 (2050) | \$/t-CO<sub>2</sub>| <a href="https://www.cell.com/joule/fulltext/S2542-4351(18)30225-3#tbl2" rel="nofollow">Keith et al. (2018)</a>|
||Energy Consumption| 8.81  |PJ Natural Gas | <a href="https://www.cell.com/joule/fulltext/S2542-4351(18)30225-3#tbl2" rel="nofollow">Keith et al. (2018)</a>,  <a href="https://pubs.rsc.org/en/content/articlelanding/2020/SE/c9se00479c#!divAbstractlife" rel="nofollow">Liu et al.</a>|

#### Geologic Sequestration and Storage <a class="anchor" id="CCS"></a>

If captured CO$_2$ (from DAC or other CCS technologies) is not transformed into synthetic fuels, it must be sequestered underground. Our database models geologic storage based on the <a href="https://acee.princeton.edu/rapidswitch/projects/net-zero-america-project/" rel="nofollow">Princeton Net-Zero America Study (2021)</a>.  In line with Net-Zero America (NZA), we model cost curves for CO$_2$ transport and storage based on capacity. We diverge from NZA in that we do not model enhanced oil recovery. We model 8 steps on the CO$_2$ storage supply curve, each with a maximum annual activity. We also model a total US storage potential of 480 gigatonnes CO$_2$ <a href="https://acee.princeton.edu/rapidswitch/projects/net-zero-america-project/" rel="nofollow">(NZA, 2021)</a>. Cumulative and annual national capacities are outlined in Table 2. We estimate regional annual capacities with NZA data, but cumulative regional storage potential is estimated using data from <a href="https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3366176" rel="nofollow">Teletzke et al (2020)</a>. Currently, we assume no CO$_2$ leakage occurs. We also do not model inter-regional CO$_2$ transport.


**Table 2.** Key National CCS parameters

| **Parameter**                                       | **Value \[units\]**                | **Source**                                                                       |
| --------------------------------------------------- | ---------------------------------- | -------------------------------------------------------------------------------- |
| Variable Cost (includes transportation and storage) | 0.0198 – 0.0703 \[$M/kt-CO<sub>2</sub>/year\] | [NZA](https://acee.princeton.edu/rapidswitch/projects/net-zero-america-project/) |
| Total US capacity                                   | 480 \[Gt CO<sub>2</sub>\]                       | [NZA](https://acee.princeton.edu/rapidswitch/projects/net-zero-america-project/) |
| Annual Capacity, step 1                             | 100,000 \[kt CO<sub>2</sub>\]                 | [NZA](https://acee.princeton.edu/rapidswitch/projects/net-zero-america-project/) |
| Annual Capacity, step 2                             | 100,000 \[kt CO<sub>2</sub>\]                 | [NZA](https://acee.princeton.edu/rapidswitch/projects/net-zero-america-project/) |
| Annual Capacity, step 3                             | 700,000 \[kt CO<sub>2</sub>\]                 | [NZA](https://acee.princeton.edu/rapidswitch/projects/net-zero-america-project/) |
| Annual Capacity, step 4                             | 500,000 \[kt CO<sub>2</sub>\]                 | [NZA](https://acee.princeton.edu/rapidswitch/projects/net-zero-america-project/) |
| Annual Capacity, step 5                             | 260,000 \[kt CO<sub>2</sub>\]                 | [NZA](https://acee.princeton.edu/rapidswitch/projects/net-zero-america-project/) |
| Annual Capacity, step 6                             | 100,000 \[kt CO<sub>2</sub>\]                 | [NZA](https://acee.princeton.edu/rapidswitch/projects/net-zero-america-project/) |
| Annual Capacity, step 7                             | 60,000 \[kt CO<sub>2</sub>\]                  | [NZA](https://acee.princeton.edu/rapidswitch/projects/net-zero-america-project/) |
| Annual Capacity, step 8                             | 40,000 \[kt CO<sub>2</sub>\]                  | [NZA](https://acee.princeton.edu/rapidswitch/projects/net-zero-america-project/) |


### Technology/commodity description lookup tool <a class="anchor" id="description_look_up"></a>
Use the tool below to search for any key words that may describe a technology or commodity of interest (e.g. heating, cooling). The tool provides a list of all the technologies or commodities in the database that may be relevant to the query.

In [5]:
w = widgets.Text(value='DAC')
display(w)
def f(w):
    if len(w)>0:
        df1 = pd.read_sql("SELECT * FROM commodities WHERE comm_desc LIKE '%" + w + "%'", con)
        df1['desc'] = df1['comm_desc'].str.replace('#','').str.strip()
        df1['comm_tech'] = df1['comm_name']
        df1['type'] = 'commodity'

        df2 = pd.read_sql("SELECT * FROM technologies WHERE tech_desc LIKE '%" + w + "%'", con)
        df2['desc'] = df2['tech_desc'].str.replace('#','').str.strip()
        df2['comm_tech'] = df2['tech']
        df2['type'] = 'technology'


        df = pd.concat([df1[['comm_tech','type','desc']], df2[['comm_tech','type','desc']]])
        
        if len(df)>0:
            display(HTML(tabulate.tabulate(df.set_index('comm_tech'),['technology/commodity','type','description'],stralign='left',tablefmt='html')))
        else:
            print('')
    else:
        print('')
            


out = widgets.interactive_output(f, {'w': w})
display(out)

Text(value='DAC')

Output()

### Network diagram lookup tool  <a class="anchor" id="network_look_up"></a>
Use the [description lookup tool](#description_look_up) above to identify specific commodity or technology names. Type the name in the box below to generate a corresponding network diagram for that commodity or technology. The slider can be used to view different upstream levels of the network diagram.

In [6]:
w = widgets.Text(value='ATM_co2_ground')
display(w)
def f(w):
    if len(w)>0:
        
        df1 = pd.read_sql("SELECT comm_name, comm_desc FROM commodities WHERE comm_name='" + w + "'", con)
        df1['desc'] = df1['comm_desc'].str.replace('#','').str.strip()
        df1['comm_tech'] = df1['comm_name']
        df1['type'] = 'commodity'

        df2 = pd.read_sql("SELECT * FROM technologies WHERE tech='" + w + "'", con)
        df2['desc'] = df2['tech_desc'].str.replace('#','').str.strip()
        df2['comm_tech'] = df2['tech']
        df2['type'] = 'technology'

        df = pd.concat([df1[['comm_tech','type','desc']], df2[['comm_tech','type','desc']]])

    
        if len(df)>0:
            def show_desc(level):
                display(Markdown(df['desc'][0]))
                final_dem = df['comm_tech'][0]
                df_graph = return_flowd_table(final_dem,level)
                args = create_args_flowd(df_graph)
                colors, quick_run_dot_fmt = return_format_colors()
                args.update(colors)
                #o_str = 'rankdir = "LR" ;'
                #r_str = 'rankdir = "LR" ; \n\t size="8,8";'
                #quick_run_dot_fmt = quick_run_dot_fmt.replace(o_str, r_str)
                dot_graph = quick_run_dot_fmt % args
                display(graphviz.Source(dot_graph))
            w2 = widgets.IntSlider(value=1,min=0,max=10,step=1,description='Level:',disabled=False,continuous_update=True,orientation='horizontal',readout=True,readout_format='d')
            w = widgets.interactive(show_desc, level=w2)
            controls_rows(w)
        else:
            print('')

    else:
        print('')
            


out = widgets.interactive_output(f, {'w': w})
display(out)

Text(value='ATM_co2_ground')

Output()